# HW 1: Corporate Bond Pricing (due by 9.17 Tues)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2019`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2019/HW1/HW1.ipynb`__  (Please use the same name for repository and ipynb file)
* Add solution code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [21]:
import numpy as np

In [22]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [23]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(9.93742116013964, 10.0)

In [24]:
# Put your code here to price the corporate bond
def corp_bond(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4):
    U = np.random.uniform(size=n_sample)
    default_time = -(1 / def_rate) * np.log(U)
    
    price_list = []
    for i in default_time:
        if i > mat:
            temp_price = 1.0 * np.exp(-rf_rate * mat)
        else:
            temp_price = recovery * 1.0 * np.exp(-rf_rate * i)
        price_list.append(temp_price)
    
    mean_price = np.mean(np.array(price_list))
    return mean_price

price = corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
print('Solution 1: The solution of calling function is', price)


price_list = []
for j in range(100):
    temp_price = corp_bond(mat, def_rate, rf_rate, recovery, n_sample)
    price_list.append(temp_price)

mean_price = np.mean(price_list)
std_price = np.std(price_list)
print('Solution 2: if we call for 100 times, \
we can get the mean is {0}, and the standard deviation is {1}.'.format(mean_price, std_price))

Solution 1: The solution of calling function is 0.4382070164229224
Solution 2: if we call for 100 times, we can get the mean is 0.44062452893315907, and the standard deviation is 0.0022896972374192026.


### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [25]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=int(n_sample/2))
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(9.999999999999996, 10.0)

In [26]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=1, def_rate=0.03, rf_rate=0.04, recovery=0.3, n_sample=1e4, antithetic=True, mean_match=True):
    ### <--
    if(antithetic):
        U = np.random.uniform(size=int(n_sample/2))
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
    else:
        U = np.random.uniform(size=n_sample)
        default_time = -(1 / def_rate) * np.log(U)
    
    if(mean_match):
        default_time += 1/def_rate-default_time.mean()
    
    price_list = []
    for i in default_time:
        if i > mat:
            temp_price = 1.0 * np.exp(-rf_rate * mat)
        else:
            temp_price = recovery * 1.0 * np.exp(-rf_rate * i)
        price_list.append(temp_price)
    
    return np.mean(price_list)


antithetic = 1
mean_match = 0
price_list = []
for j in range(100):
    temp_price = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,\
                        antithetic=antithetic, mean_match=mean_match)
    price_list.append(temp_price)

mean_price = np.mean(price_list)
std_price = np.std(price_list)
print('Solution 3: Via antithetic method, \
the mean is {0}, and the standard deviation is {1}.'.format(mean_price, std_price))


antithetic = 0
mean_match = 1
price_list = []
for j in range(100):
    temp_price = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,\
                        antithetic=antithetic, mean_match=mean_match)
    price_list.append(temp_price)

mean_price = np.mean(price_list)
std_price = np.std(price_list)
print('Solution 4: Via mean_match method, \
the mean is {0}, and the standard deviation is {1}.'.format(mean_price, std_price))

#(ii)both
antithetic = 1
mean_match = 1
price_list = []
for j in range(100):
    price = corp_bond_cv(mat, def_rate, rf_rate, recovery, n_sample,\
                        antithetic=antithetic, mean_match=mean_match)
    price_list.append(price)

mean_price = np.mean(price_list)
std_price = np.std(price_list)
print('Solution 5: Combining the antithetic method and mean_match method, \
the mean is {0}, and the standard deviation is {1}.'.format(mean_price, std_price))

Solution 3: Via antithetic method, the mean is 0.44044466202804544, and the standard deviation is 0.0017208452914096296.
Solution 4: Via mean_match method, the mean is 0.44042619543125655, and the standard deviation is 0.0014959917212545386.
Solution 5: Combining the antithetic method and mean_match method, the mean is 0.4406495656012867, and the standard deviation is 0.0013674693584874078.


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [27]:
### Put the analytic expression for the corporate bond price
r = def_rate + rf_rate
analytic_price = 0.3 * (1 - np.exp(-r*mat)) * def_rate / r + np.exp(-r * mat)
print('Solution 6: The analytic price is', analytic_price)
print('Solution 7: The MC result combining both antithetic method and mean_match \
method is', mean_price)
print('Solution 8: The difference is', analytic_price - mean_price)

Solution 6: The analytic price is 0.44040907156462505
Solution 7: The MC result combining both antithetic method and mean_match method is 0.4406495656012867
Solution 8: The difference is -0.0002404940366616537
